# Architectural Principles

- Principals are guides, not rules

## Separation of concerns (SoC)

- Seperate software into logic components 
- Applies to all levels of design and architecture

## Don’t repeat yourself (DRY)

- Eliminate redunancy 
- Encapsulate duplicate logic into a reusable component
- Keep SoC principal in mind when refactoring for DRY

## Keep it simple, stupid (KISS)

- Advocates for simple designs and solutions
- Choose the simplest solution possible
- Eliminate complexity
- Predicting future requirments can be detrimental to design (Over Architecting)

## You Aren’t Gonna Need It (YAGNI)

- Do not add features or functionality based on speculation
- Design the minimal amount needed to meet requirments
- Relates to the KISS principal
- Be agile and flexible to intermittently adapt and evolve as needs emerge

## The SOLID principles

SOLID is an acronym for 5 principles

### Single responsibility principle

- A single class should hold 1 and only 1 responsibility (Only 1 reason to change)

    In this example, Public and Prviate projects are 2 responsibilities and can be seperated into `PulicProdctReader` and `PrivateProductReader` classes.

    ```csharp
    namespace BeforeSRP;
    public class ProductRepository
    {
    public ValueTask<Product> GetOnePublicProductAsync(int productId)
    => throw new NotImplementedException();
    public ValueTask<Product> GetOnePrivateProductAsync(int productId)
    => throw new NotImplementedException();
    public ValueTask<IEnumerable<Product>> GetAllPublicProductsAsync()
    => throw new NotImplementedException();
    public ValueTask<IEnumerable<Product>> GetAllPrivateProductsAsync()
    => throw new NotImplementedException();
    public ValueTask CreateAsync(Product product)
    => throw new NotImplementedException();
    public ValueTask UpdateAsync(Product product)
    => throw new NotImplementedException();
    public ValueTask DeleteAsync(Product product)
    => throw new NotImplementedException();
    }
    ```
- Be careful not to over-seperate becuase that adds complexity too
- Trouble naming a class may indicate a problem with SoC or SRP
- Another indicator is that a method or class is too big

### Open/closed principle

> ”Software entities (classes, modules, functions, and so on) should be open for extension 
but closed for modification.”

- Composition over inheritance
    * Compose classes of reusable components rather than inheriting methods
    * "Is a" vs "Has a" relationship

    EntityService is tightly coupled to EntityRepository

    ```csharp
    namespace OCP.NoComposability;
    public class EntityService : EntityRepository
    {
        public async Task ComplexBusinessProcessAsync(Entity entity)
        {
            // Do some complex things here
            await CreateAsync(entity);
            // Do more complex things here
        }
    }
    ```

    Composable method

    ```csharp
    namespace OCP.Composability;
    public class EntityService
    {
        private readonly EntityRepository _repository = new EntityRepository();

        public async Task ComplexBusinessProcessAsync(Entity entity)
        {
            // Do some complex things here
            await _repository.CreateAsync(entity);
            // Do more complex things here
        }
    }
    ```

- Depency injection

    ```csharp
    namespace OCP.DepencyInjection;
    public class EntityService
    {
        private readonly EntityRepository _repository;

        public EntityService(EntityRepository repository)
        {
            _repository = respository;
        }

        public async Task ComplexBusinessProcessAsync(Entity entity)
        {
            // Do some complex things here
            await _repository.CreateAsync(entity);
            // Do more complex things here
        }
    }
    ```

### Liskov substitution principle

> The LSP states that in a program, if we replace an instance of a superclass (supertype) with an instance 
of a subclass (subtype), the program should not break or behave unexpectedly

> In simpler words, if S is a subtype of T, we can replace objects of type T with objects of type S without 
changing any of the expected behaviors of the program (correctness)

- Apply when using inheritance
- Complex requirement to meet it
- should be much of a problem when using composition over inheritance
- On requirement is that a subtype must pass all tests written for a supertype

> The key idea of the LSP is that the consumer of a supertype should remain unaware of whether it’s 
interacting with an instance of a supertype or an instance of a subtype.

### Interface segregation principle

- More small interfaces are better than fewer large interfaces
- An interface defines a cohesive contract (methods, properties, and events)
- Since C# 8 you can define a default implementation of a method in an interface

### Dependency inversion principle

- Use abrstractions (Interfaces) to reduce dependencies between classes

Instead of this:

In [36]:
classDiagram
    SomeService --> LocalDataPersistance: Uses
    SomeService --> SqlDataPersistance: Uses

Do this:

> We can achieve this by introducing an abstraction (like an interface) between the modules This means 
that instead of Class A depending directly on Class B, Class A would rely on an abstraction that Class 
B implements.

In [ ]:
classDiagram
    SomeService --> IDataPersistance: Uses
    IDataPersistance <|-- LocalDataPersistance
    IDataPersistance <|-- SqlDataPersistance


These abstractions can be moved into assemblies for subsystems as well to help decouple at a higher level

In [39]:
classDiagram
namespace Core {
    class SomeService
}

namespace Abstractions {
    class IDataPersistance
}

namespace Local {
    class LocalDataPersistance
}
namespace Sql {
    class SqlDataPersistance
}
    SomeService --> IDataPersistance
    IDataPersistance <|-- LocalDataPersistance
    IDataPersistance <|-- SqlDataPersistance

Example project at `C03\Dependency Inversion`. 

In [40]:
public interface IDataPersistence
{
    void Persist();
}

public class SqlDataPersistence : IDataPersistence
{
    public void Persist()
    {
        Console.WriteLine("Data persisted by SqlDataPersistence.");
    }
}

public class LocalDataPersistence : IDataPersistence
{
    public void Persist()
    {
        Console.WriteLine("Data persisted by LocalDataPersistence.");
    }
}

public class SomeService
{
    public void Operation(IDataPersistence someDataPersistence)
    {
        // The someDataPersistence instance is responsible
        // for the location where the data is persisted.
        Console.WriteLine("Beginning SomeService.Operation.");
        someDataPersistence.Persist();
        Console.WriteLine("SomeService.Operation has ended.");
    }
}

var sqlDataPersistence = new SqlDataPersistence();
var localDataPersistence = new LocalDataPersistence();

var service = new SomeService();
service.Operation(localDataPersistence);
service.Operation(sqlDataPersistence);

Beginning SomeService.Operation.
Data persisted by LocalDataPersistence.
SomeService.Operation has ended.
Beginning SomeService.Operation.
Data persisted by SqlDataPersistence.
SomeService.Operation has ended.
